Toronto, despite being a major city in the world, is often viewed as boring. I believe that this is due to the lack of after-work activities provided. I hope to dive deeper into this problem, create a feasible solution, and maybe even put it into action.



If there are actually a lack of 'fun' things to do in Toronto, certain metrics will be able to determine if this is true or not.
Some questions I hope to find the answer to using the data are:
    
Compared to other major cities, what kind of venues are lacking? (A follow up question to that is: why are they lacking? This will require more research outside of the foursquare data and may depend on laws that affect alcohol distribution, or are dependent on the cost of living, etc.)

Once deciding on the venue to pursue, I will have to decide where it will be located. I can scrape the internet for the rent prices, income levels, population density, that match the type of business I want to start. I would also like to look into accessability going to and from the venue. At this stage, I would have to look into other metrics that factor into if customers were to come to that area or that type of venue.

In [128]:
CLIENT_ID = 'Y1PZQ4UQX0XRYB5YMWSHH05PWS2F0R4S4SI4F05KIUJZ1JDX' # your Foursquare ID
CLIENT_SECRET = 'FFMOJY1UQXST5XXCTUIGWVLENGPG2VDDJIYF201YLOXUTG30' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [108]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

/bin/sh: conda: command not found
/bin/sh: conda: command not found
Libraries imported.


In [144]:
import requests
import lxml.html as lh
import geocoder

In [145]:
url='https://en.wikipedia.org/wiki/List_of_cities_by_international_visitors'

In [146]:
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [147]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]

In [148]:
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:%s'%(i,name))
    col.append((name,[]))

1:RankEuromonitor

2:RankMastercard

3:City

4:Country

5:Arrivals 2017Euromonitor

6:Arrivals 2016Mastercard

7:Growthin arrivalsEuromonitor

8:Income(billions $)Mastercard



In [149]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=8:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [150]:
#Check the length of each column
[len(C) for (title,C) in col]

[137, 137, 137, 137, 137, 137, 137, 137]

In [151]:
#Create dictionary and DataFrame
Dict={title:column for (title,column) in col}
df_raw=pd.DataFrame(Dict)
df_raw.head()

,RankEuromonitor\n,RankMastercard\n,City\n,Country\n,Arrivals 2017Euromonitor\n,Arrivals 2016Mastercard\n,Growthin arrivalsEuromonitor\n,Income(billions $)Mastercard\n
0,1\n,11,Hong Kong\n,Hong Kong\n,"25,695,800\n","8,370,000\n",−3.1 %\n,6.84\n
1,2\n,1,Bangkok\n,Thailand\n,"23,270,600\n","21,470,000\n",9.5 %\n,14.84\n
2,3\n,2,London\n,United Kingdom\n,"19,842,800\n","19,880,000\n",3.4 %\n,19.76\n
3,4\n,6,Singapore\n,Singapore\n,"17,681,800\n","12,110,000\n",6.1 %\n,12.54\n
4,5\n,\n,Macau\n,Macau\n,"16,299,100\n",\n,5.9 %\n,\n


In [152]:
#Clean up the DataFrame
df = df_raw
df.rename(columns={'RankEuromonitor\n':'RankMastercard','City\n':'City','Country\n':'Country','Arrivals 2017Euromonitor\n':'Arrivals','Growthin arrivalsEuromonitor\n':'Growth'}, inplace=True)
df = df.replace('\n','', regex=True) #get rid of all \n in the DataFrame
df = df.replace('\xa0','', regex=True) #get rid of all \xa0 in the DataFrame
df = df.replace('',np.nan, regex=True) #replace "Not Assigned" values to NaN
print(df['City'].values[:10])
df.drop(['RankMastercard\n','Arrivals 2016Mastercard\n','Income(billions $)Mastercard\n'], axis=1,inplace=True)
df.dropna(inplace=True) #drop rows that have NaN
df['City, Country'] = df['City'] + ', ' + df['Country']
df.head(10)

['Hong Kong' 'Bangkok' 'London' 'Singapore' 'Macau' 'Dubai' 'Paris'
 'New York City' 'Shenzhen' 'Kuala Lumpur']


,RankMastercard,City,Country,Arrivals,Growth,"City, Country"
0,1,Hong Kong,Hong Kong,"25,695,800",−3.1 %,"Hong Kong, Hong Kong"
1,2,Bangkok,Thailand,"23,270,600",9.5%,"Bangkok, Thailand"
2,3,London,United Kingdom,"19,842,800",3.4%,"London, United Kingdom"
3,4,Singapore,Singapore,"17,681,800",6.1%,"Singapore, Singapore"
4,5,Macau,Macau,"16,299,100",5.9%,"Macau, Macau"
5,6,Dubai,United Arab Emirates,"16,010,000",7.7%,"Dubai, United Arab Emirates"
6,7,Paris,France,"14,263,000",−0.9%,"Paris, France"
7,8,New York City,United States,"13,100,000",3.6%,"New York City, United States"
8,9,Shenzhen,China,"12,962,000",3.1%,"Shenzhen, China"
9,10,Kuala Lumpur,Malaysia,"12,843,500",4.5%,"Kuala Lumpur, Malaysia"


In [168]:
Cities = df['City, Country'][:50].values
Cities

array(['Hong Kong, Hong Kong', 'Bangkok, Thailand',
       'London, United Kingdom', 'Singapore, Singapore', 'Macau, Macau',
       'Dubai, United Arab Emirates', 'Paris, France',
       'New York City, United States', 'Shenzhen, China',
       'Kuala Lumpur, Malaysia', 'Phuket, Thailand', 'Delhi, India',
       'Tokyo, Japan', 'Rome, Italy', 'Taipei, Taiwan',
       'Guangzhou, China', 'Mumbai, India', 'Mecca, Saudi Arabia',
       'Istanbul, Turkey', 'Prague, Czech Republic',
       'Miami, United States', 'Seoul, South Korea', 'Barcelona, Spain',
       'Pattaya, Thailand', 'Shanghai, China', 'Milan, Italy',
       'Cancún, Mexico', 'Agra, India', 'Las Vegas, United States',
       'Amsterdam, Netherlands', 'Antalya, Turkey', 'Denpasar, Indonesia',
       'Osaka, Japan', 'Los Angeles, United States', 'Vienna, Austria',
       'Berlin, Germany', 'Madrid, Spain', 'Johor Bahru, Malaysia',
       'Johannesburg, South Africa', 'Riyadh, Saudi Arabia',
       'Ho Chi Minh City, Vietnam', '

In [177]:
df_coordinates = []
for city in Cities:
    geolocator = Nominatim(user_agent="explorer")
    location = geolocator.geocode(city, timeout=10)
    latitude = location.latitude
    longitude = location.longitude
    df_coordinates.append([city, latitude, longitude])
print('Done!')

In [181]:
df_coordinates[:5]

[['Hong Kong, Hong Kong', 22.350627, 114.1849161],
 ['Bangkok, Thailand', 13.7538929, 100.8160803],
 ['London, United Kingdom', 51.5073219, -0.1276474],
 ['Singapore, Singapore', 1.3408528, 103.878446863736],
 ['Macau, Macau', 22.1956294, 113.548784711485]]

In [182]:
cols = ['City, Country','Latitude','Longitude']
df1 = pd.DataFrame(df_coordinates, columns=cols)
df1.head()


,"City, Country",Latitude,Longitude
0,"Hong Kong, Hong Kong",22.350627,114.184916
1,"Bangkok, Thailand",13.753893,100.816080
2,"London, United Kingdom",51.507322,-0.127647
3,"Singapore, Singapore",1.340853,103.878447
4,"Macau, Macau",22.195629,113.548785


In [191]:
df2 = pd.merge(df1, df, on=['City, Country'])
df2.head(1)

,"City, Country",Latitude,Longitude,RankMastercard,City,Country,Arrivals,Growth
0,"Hong Kong, Hong Kong",22.350627,114.184916,1,Hong Kong,Hong Kong,"25,695,800",−3.1 %


In [203]:
df2 = df2[['RankMastercard','City, Country','Arrivals','Growth','Latitude','Longitude']]
df2.head()

,RankMastercard,"City, Country",Arrivals,Growth,Latitude,Longitude
0,1,"Hong Kong, Hong Kong","25,695,800",−3.1 %,22.350627,114.184916
1,2,"Bangkok, Thailand","23,270,600",9.5%,13.753893,100.816080
2,3,"London, United Kingdom","19,842,800",3.4%,51.507322,-0.127647
3,4,"Singapore, Singapore","17,681,800",6.1%,1.340853,103.878447
4,5,"Macau, Macau","16,299,100",5.9%,22.195629,113.548785
